<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_termin10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 10

Vežba obuhvata zadatke:
1.   Zadatak 1 - Histogram
2.   Zadatak 2 - Tekst histogram

## Preduslovi
Pre nego započnete implementaciju zadataka, **obavezno** izvršite kod iz ćelije ispod. Kod će:
- skinuti kompresovanu biblioteku `libwb` koja se koristi u zadacima i raspakovati je na putanju /usr/local/libwb.
- skinuti set podataka za testiranje rešenja zadataka i raspakovati ga na putanje /home/cuda/histogram i /home/cuda/text_histogram za zadatke `Histogram` i `Text Histogram` redom.

In [ ]:
%%bash

######################### PREUZMI I PODESI libwb ##############################
# preuzmi libwb.zip ako već nije preuzet
[[ -f libwb.zip ]] || wget -O libwb.zip "https://drive.google.com/u/0/uc?id=1-j31qXJJOBFKuNc4qzTjTU-Gji8HARh0&export=download"

# otpakuj
unzip -oqqd /usr/local libwb.zip


################### PREUZMI I PODESI set za testiranje ########################
# preuzmi dataset.zip za zadatak HISTOGRAM ako vec nije preuzet
[[ -f dataset_hist.zip ]] || wget -O dataset_hist.zip "https://drive.google.com/u/0/uc?id=1z0yagOeqWzchmnrC3uLa72brYJHaoqOy&export=download"

# otpakuj u direktorijum u kojem ce se nalaziti izvrsna datoteka resenja
mkdir -p /home/cuda/histogram
unzip -oqqd /home/cuda/histogram dataset_hist.zip

# preuzmi dataset.zip za zadatak TEKST HISTOGRAM ako vec nije preuzet
[[ -f dataset_thist.zip ]] || wget -O dataset_thist.zip "https://drive.google.com/u/0/uc?id=1LpGqdH0LhLSaq0V1pd_8c2xpDXRXLrWi&export=download"

# otpakuj u direktorijum u kojem ce se nalaziti izvrsna datoteka resenja
mkdir -p /home/cuda/text_histogram
unzip -oqqd /home/cuda/text_histogram dataset_thist.zip

Po završetku prethodne ćelije, možete pokrenuti narednu koja proverava da li `libwb` i `dataset` direktorijumi postoje na predviđenim putanjama. Ukoliko je sve u redu, obe naredbe će ispisati absolutne putanje do direktorijuma. U suprotnom će prijaviti da jedan ili oba direktorijuma nisu pronađena. U tom slučaju treba da se vratite na log prethodne ćelije i potražite gde je došlo do greške.

In [ ]:
%%bash

ls -d /usr/local/libwb/
ls -d /home/cuda/histogram/dataset/
ls -d /home/cuda/text_histogram/dataset/

## [Opciono] Deinstalacija preduslova
Izvršavanjem ćelije ispod brišete instaliranu biblioteku `libwb` i set podataka. Izvršite ovu ćeliju samo ako želite da resetujete instalaciju kako biste reinstalirali preduslove.

In [ ]:
%%bash

# obrisi libwb biblioteku
[[ -d /usr/local/libwb ]] && rm -r /usr/local/libwb/ && echo "Obrisana libwb biblioteka."
[[ -f libwb.zip ]] && rm libwb.zip && echo "Obrisana libwb.zip arhiva."

# obrisi setove podataka za testiranje
[[ -d /home/cuda/histogram/dataset ]] && rm -r /home/cuda/histogram/dataset && \
    echo "Obrisan set podataka za zadatak HISTOGRAM."
[[ -d /home/cuda/text_histogram/dataset ]] && rm -r /home/cuda/text_histogram/dataset && \
    echo "Obrisan set podataka za zadatak TEKST HISTOGRAM."
[[ -f dataset_hist.zip ]] && rm dataset_hist.zip && echo "Obrisana dataset.zip arhiva."
[[ -f dataset_thist.zip ]] && rm dataset_thist.zip && echo "Obrisana dataset.zip arhiva."
[[ -f dataset.zip ]] && rm dataset.zip && echo "Obrisana dataset.zip arhiva."

## Zadatak 1 - Histogram

Implementirati kernel za efikasno računanje histograma za zadati niz celobrojnih ulaznih vrednosti u datom opsegu. Svaka vrednost iz niza se preslikava na jedan od podeoka histograma. Podeoci histograma ne mogu preći vrednost veću od 127 (dostižu saturaciju na vrednosti 127). Vrednosti u ulaznom nizu za ovu vežbu mogu biti u rasponu 0 do 4096. Za predstavljanje podeoka histograma koristiti neoznačene 32-bitne brojače. Smatrati da ulazni niz neće biti duži od $2^{32}$.

Zadatak može biti podeljen u dva kernela: jedan koji računa histogram bez saturacije i drugi koji koriguje vrednosti podeoka. Ove dve faze takođe mogu biti implementirane u okviru jednog kernela. Poželjno je implementirati obe verzije kernela i uporediti performanse.

Dopuniti zadatu postavku zadatka implementacijom nedostajućih delova
koda. Celokupno rešenje treba da se nađe u ćeliji ispod. Implementirati:

-   alokaciju memorije na uređaju,

-   kopiranje podataka iz memorije domaćina u memoriju uređaja,

-   inicijalizaciju dimenzija mreže i bloka,

-   poziv kernela,

-   kopiranje podataka sa memorije uređaja u memoriju domaćina i

-   dealokaciju memorije uređaja,

-   kernel za računanje histograma

In [ ]:
%%writefile /home/cuda/histogram/histogram.cu

#include <wb.h>

#define NUM_BINS 4096

#define CUDA_CHECK(ans)                                                   \
  { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line,
                      bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code),
            file, line);
    if (abort)
      exit(code);
  }
}

int main(int argc, char *argv[]) {
  wbArg_t args;
  int inputLength;
  unsigned int *hostInput;
  unsigned int *hostBins;
  unsigned int *deviceInput;
  unsigned int *deviceBins;

  args = wbArg_read(argc, argv);

  wbTime_start(Generic, "Importing data and creating memory on host");
  hostInput = (unsigned int *)wbImport(wbArg_getInputFile(args, 0),
                                       &inputLength, "Integer");
  hostBins = (unsigned int *)malloc(NUM_BINS * sizeof(unsigned int));
  wbTime_stop(Generic, "Importing data and creating memory on host");

  wbLog(TRACE, "The input length is ", inputLength);
  wbLog(TRACE, "The number of bins is ", NUM_BINS);

  wbTime_start(GPU, "Allocating GPU memory.");
  //@@ Allocate GPU memory here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(GPU, "Allocating GPU memory.");

  wbTime_start(GPU, "Copying input memory to the GPU.");
  //@@ Copy memory to the GPU here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(GPU, "Copying input memory to the GPU.");

  // Launch kernel
  // ----------------------------------------------------------
  wbLog(TRACE, "Launching kernel");
  wbTime_start(Compute, "Performing CUDA computation");
  //@@ Perform kernel computation here
  wbTime_stop(Compute, "Performing CUDA computation");

  wbTime_start(Copy, "Copying output memory to the CPU");
  //@@ Copy the GPU memory back to the CPU here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(Copy, "Copying output memory to the CPU");

  wbTime_start(GPU, "Freeing GPU Memory");
  //@@ Free the GPU memory here
  wbTime_stop(GPU, "Freeing GPU Memory");

  // Verify correctness
  // -----------------------------------------------------
  wbSolution(args, hostBins, NUM_BINS);

  free(hostBins);
  free(hostInput);
  
  return 0;
}

### Kompajliranje izvornog koda
Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/cuda/histogram
nvcc histogram.cu -o histogram \
  -I /usr/local/libwb/ \
  -L /usr/local/libwb/lib \
  -lwb -lcuda

### [Opciono] Provera da li postoji izvrsna datoteka

In [ ]:
![[ -f /home/cuda/histogram/histogram ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja nad jednim primerom

In [ ]:
%%bash

cd /home/cuda/histogram

LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./histogram \
      -i dataset/0/input.raw \
      -o output.raw \
      -e dataset/0/output.raw \
      -t integral_vector

### Pokretanje rešenja nad svim primerima

Nakon uspešnog testiranja rešenja na jednom primeru, možete pokrenuti i testiranje rešenja na svim primerima iz seta podataka.

In [ ]:
%%bash

cd /home/cuda/histogram

n=$(ls -dl dataset/* | wc -l)

for (( i=0; i<$n; i=i+1 ))
do
  echo "Testiram primer $i."
  echo "-------------------"
  LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./histogram \
      -i dataset/$i/input.raw \
      -o output.raw \
      -e dataset/$i/output.raw \
      -t integral_vector
  echo ""
done

## Zadatak 2 - Text Histogram

Implementirati kernel za računanje histograma ulaznog niza ASCII karaktera. Ukupno ima 128 ASCII karaktera, koji se mapiraju na odgovarajuće podeoke histograma (kojih takođe ima 128). Podeoci histograma su predstavljeni 32-bitnim neoznačenim vrednostima koje ne saturiraju. Pri implementaciji zadatka za svaki blok niti kreirati privatni histogram. Koristeći atomične operacije modifikovati globalni histogram u skladu sa privatnim histogramima.

Dopuniti zadatu postavku zadatka implementacijom nedostajućih delova
koda. Celokupno rešenje treba da se nađe u ćeliji ispod. Implementirati:

-   alokaciju memorije na uređaju,
-   kopiranje podataka iz memorije domaćina u memoriju uređaja,
-   inicijalizaciju dimenzija mreže i bloka,
-   poziv kernela,
-   kopiranje podataka sa memorije uređaja u memoriju domaćina i
-   dealokaciju memorije uređaja,
-   kernel za računanje histograma

In [ ]:
%%writefile /home/cuda/text_histogram/text_histogram.cu

#include <wb.h>

#define NUM_BINS 128

#define CUDA_CHECK(ans)                                                   \
  { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line,
                      bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code),
            file, line);
    if (abort)
      exit(code);
  }
}

int main(int argc, char *argv[]) {
  wbArg_t args;
  int inputLength;
  char *hostInput;
  unsigned int *hostBins;
  char *deviceInput;
  unsigned int *deviceBins;

  args = wbArg_read(argc, argv);

  wbTime_start(Generic, "Importing data and creating memory on host");
  hostInput = (char *)wbImport(wbArg_getInputFile(args, 0),
                                       &inputLength, "Text");
  hostBins = (unsigned int *)malloc(NUM_BINS * sizeof(unsigned int));
  wbTime_stop(Generic, "Importing data and creating memory on host");

  wbLog(TRACE, "The input length is ", inputLength);
  wbLog(TRACE, "The number of bins is ", NUM_BINS);

  wbTime_start(GPU, "Allocating GPU memory.");
  //@@ Allocate GPU memory here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(GPU, "Allocating GPU memory.");

  wbTime_start(GPU, "Copying input memory to the GPU.");
  //@@ Copy memory to the GPU here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(GPU, "Copying input memory to the GPU.");

  // Launch kernel
  // ----------------------------------------------------------
  wbLog(TRACE, "Launching kernel");
  wbTime_start(Compute, "Performing CUDA computation");
  //@@ Perform kernel computation here
  wbTime_stop(Compute, "Performing CUDA computation");

  wbTime_start(Copy, "Copying output memory to the CPU");
  //@@ Copy the GPU memory back to the CPU here
  CUDA_CHECK(cudaDeviceSynchronize());
  wbTime_stop(Copy, "Copying output memory to the CPU");

  wbTime_start(GPU, "Freeing GPU Memory");
  //@@ Free the GPU memory here
  wbTime_stop(GPU, "Freeing GPU Memory");

  // Verify correctness
  // -----------------------------------------------------
  wbSolution(args, hostBins, NUM_BINS);

  free(hostBins);
  free(hostInput);
  
  return 0;
}

### Kompajliranje izvornog koda
Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/cuda/text_histogram
nvcc text_histogram.cu -o text_histogram \
  -I /usr/local/libwb/ \
  -L /usr/local/libwb/lib \
  -lwb -lcuda

### [Opciono] Provera da li postoji izvrsna datoteka

In [ ]:
![[ -f /home/cuda/text_histogram/text_histogram ]] && echo "Postoji." || echo "Ne postoji." # TODO izmeni

### Pokretanje rešenja nad jednim primerom

In [ ]:
%%bash

cd /home/cuda/text_histogram
LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./text_histogram \
      -i dataset/0/input.txt \
      -o output.raw \
      -e dataset/0/output.raw \
      -t integral_vector

### Pokretanje rešenja nad svim primerima

Nakon uspešnog testiranja rešenja na jednom primeru, možete pokrenuti i testiranje rešenja na svim primerima iz seta podataka.

In [ ]:
%%bash

cd /home/cuda/text_histogram

n=$(ls -dl dataset/* | wc -l)

for (( i=0; i<$n; i=i+1 ))
do
  echo "Testiram primer $i."
  echo "-------------------"
  LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./text_histogram \
      -i dataset/0/input.txt \
      -o output.raw \
      -e dataset/0/output.raw \
      -t integral_vector                                                          
  echo ""
done